In [5]:
import ROOT as r

from pathlib import Path

import pickle
import json

In [19]:
tablica = {}

#with open("Signal_efficiency.pickle", "wb") as f:
    #pickle.dump(tablica, f)

{}

In [10]:
with open("Signal_efficiency_newcuts.pickle", "rb") as f:
    tab = pickle.load(f)
tab

{'400, 260': 0.0001,
 '800, 260': 0.0001,
 '800, 300': 0.0004,
 '800, 660': 0.0006209251785159888,
 '1200, 260': 0.002004208838560978,
 '1200, 300': 0.0016207455429497568,
 '1200, 700': 0.0112,
 '1200, 1060': 0.0028,
 '1600, 260': 0.001,
 '1600, 300': 0.0019120458891013384,
 '1600, 700': 0.033,
 '1600, 1100': 0.0536,
 '1600, 1460': 0.004314236982040734,
 '2000, 260': 0.0009,
 '2000, 300': 0.0008,
 '2000, 700': 0.0556,
 '2000, 1100': 0.092,
 '2000, 1500': 0.0713,
 '2000, 1860': 0.0042,
 '2400, 260': 0.0012,
 '2400, 300': 0.0006,
 '2400, 700': 0.0677,
 '2400, 1100': 0.0969,
 '2400, 1500': 0.1212,
 '2400, 1900': 0.0761,
 '2400, 2260': 0.0062,
 '2800, 260': 0.0008,
 '2800, 300': 0.0012112647622892904,
 '2800, 700': 0.0667,
 '2800, 1100': 0.1042,
 '2800, 1500': 0.1271,
 '2800, 1900': 0.1344,
 '2800, 2300': 0.0774,
 '2800, 2660': 0.0071,
 '3200, 260': 0.0017582417582417582,
 '3200, 300': 0.0011014318614198458,
 '3200, 700': 0.058,
 '3200, 1100': 0.1058,
 '3200, 1500': 0.1315,
 '3200, 1900': 

In [7]:
def efficiencies(min_M2, max_M2, min_M3, max_M3):
    tablica = {}
    
    for i in range(min_M3, max_M3+400, 400):
        m3 = str(i)
        for j in range(min_M2, max_M2+10, 10):
            signals = 0
            m2 = str(j)
            
                
            Filename = ("/STORE/ferencek/TRSM_XToHY_6b/2017/13TeV/NANOAOD/TRSM_XToHY_6b_M3_"
                        + m3 +
                        "_M2_"
                        + m2 +
                        "_NANOAOD.root")

            if Path(Filename).is_file():


                f = r.TFile.Open(Filename)
                tree = f.Get("Events")

                nEvts = tree.GetEntriesFast()

                for k in range(nEvts):    
                    tree.GetEvent(k)


                    if tree.nFatJet<3:
                        continue

                    pts     = tree.FatJet_pt
                    etas    = tree.FatJet_eta
                    phis    = tree.FatJet_phi
                    masses  = tree.FatJet_msoftdrop
                    
                    with open("cuts.json", "r") as cuts:
                        cuts_dict = json.load(cuts)
                        
                    cuts_list = list(cuts_dict.values())
                    
                    cut_pts = cuts_list[0]
                    cut_etas = cuts_list[1]
                    cut_masses_min = cuts_list[2]
                    cut_masses_max = cuts_list[3]
    
    

                    jet0Flag = pts[0]>cut_pts and abs(etas[0])<cut_etas  and masses[0]>cut_masses_min and masses[0]<cut_masses_max
                    jet1Flag = pts[1]>cut_pts and abs(etas[1])<cut_etas  and masses[1]>cut_masses_min and masses[1]<cut_masses_max
                    jet2Flag = pts[2]>cut_pts and abs(etas[2])<cut_etas  and masses[2]>cut_masses_min and masses[2]<cut_masses_max

                    if (jet0Flag and jet1Flag and jet2Flag):
                        signals += 1
                eff = signals/nEvts
                tablica.update({m3 + ", " + m2: eff})
                with open("Signal_efficiency_newcuts.pickle", "wb") as f:
                    pickle.dump(tablica, f)

                
            
                    
                
        
    return "Finished calculating. Use the pickle mode to access the results"

In [8]:
%%time
efficiencies(min_M2 = 0, max_M2 = 4000, min_M3 = 0, max_M3=4000)

CPU times: user 7min 8s, sys: 9.2 s, total: 7min 17s
Wall time: 9min 14s


'Finished calculating. Use the pickle mode to access the results'

In [26]:
%%time

Filename = "/STORE/ferencek/TRSM_XToHY_6b/2017/13TeV/NANOAOD/TRSM_XToHY_6b_M3_2000_M2_1100_NANOAOD.root"

f = r.TFile.Open(Filename)
tree = f.Get("Events")

nEvts = tree.GetEntriesFast()

signals = 0
    
for i in range(nEvts): 
    tree.GetEvent(i)


    if tree.nFatJet<3:
        continue

    pts     = tree.FatJet_pt
    etas    = tree.FatJet_eta
    phis    = tree.FatJet_phi
    masses  = tree.FatJet_msoftdrop

    jet0Flag = pts[0]>250 and abs(etas[0])<2.5 and masses[0]>60
    jet1Flag = pts[1]>250 and abs(etas[1])<2.5 and masses[1]>60
    jet2Flag = pts[2]>250 and abs(etas[2])<2.5 and masses[2]>60

    if (jet0Flag and jet1Flag and jet2Flag):
        signals += 1

eff = signals/nEvts
eff

CPU times: user 6.1 s, sys: 58 ms, total: 6.16 s
Wall time: 6.14 s


0.2287